Template code structure for the pre-processing steps to be done by the end user.


- for spectral similarity scoring only the ms2query folder location is needed in the current implementation. models are automatically fetched.
- ms2query's currently used implementation can only do batch folder runs. If the single file run is available, the input-filename.mgf can be used for both ms2query and the matchms mgf importer. For now, the >> assumed << single mgf file input name is needed separately from the folder name.

- mds2query provides some convenience with folder based batch processing. My preferred approach for this is different however. I'd like to have a batch processing tool which can merge mutliple mgf files into one, but then processing is kept to a single linear input -> output chain whithout different numbers of intermediates. I process the standards differently from the experimental mgf file since the annotation approach differs.

matchms notes
- save_as_mgf does not overwrite an existing mgf; instead it starts to add to it! There is no mention of this anywhere of course.

ms2query notes:
- the ms2query models and library folder will contain the models for pairwise similarity matrix computation
- however, they will be multiple files, with complex names, including post vs neg differentiation.
- ms2query files need to be downloaded to two separate folders for positive and negative modes
- note also that ms2query seems to produce a different results.csv for negative mode than for positvie mode (neg has no classificaiton); using the classification table from positive mode seems to work however
- The version of ms2query currently used does not have support for single input file runs using run_ms2query_single_file (not defined)
- the zenodo downloader avoids downloading if files are already present. Assessment of filenames is done using the zenodo directory provided and the various filenames contained within the folder.
- ms2query filenames are long, timestamped and quite inconsistent. Writing generic find and load model files may be tricky. ms2ds and spec2vec / s2v occur in names. In general, in the ion mode specific folder, one .hdf5 file for ms2deepscore needs to be loaded. In addition, a .model, a .model.syn1neg.npy, and a model.wv.vectors.npy need to be loaded. On the basis of file type suffixes alone the corresponding files could thus be identified and loaded. In addition, spec2vec loading only requires the .model path, the rest is dealt with automatically. Hence, in each model case, only one file needs to be found. 
- ms2query produces a result csv file for each spectrums.mgf file in the input file location. It inherits the name of the input.mgf file, with .mgf being turned into .csv. 
- Filenames for standards and experimental spectra should be different to avoid filename overlap problems from ms2query's folder processing approach


- ms2query will refuse to give any results for some spetra that fail to pass some cleaning criteria. Those spectra will be omitted from the results.csv file. However, the ms2query query_spectrum_nr will be based on the iloc + 1 of the spectra in the original list. This means that it is in principle possible to expand the reduced size ms2query output to full size with NA.

In [1]:
from template_processing_funs import clean_spectra
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf


In [2]:

spectrums_exp = list(load_from_mgf("./testing/raw_data/spectra_set1.mgf"))
n_raw = len(spectrums_exp)
print(n_raw)
spectrums_exp = clean_spectra(spectrums_exp)
spectrums_exp = list(filter(lambda item: item is not None, spectrums_exp))
n_cleaned = len(spectrums_exp)
print(n_cleaned)
print(f"Filtered out {n_raw - n_cleaned} of a total of {n_raw} spectra. {n_cleaned} spectra remaining.")


spectrums_standards = list(load_from_mgf("./testing/raw_data/spectra_set2.mgf"))
n_raw = len(spectrums_standards)
print(n_raw)
spectrums_standards = clean_spectra(spectrums_standards)
spectrums_standards = list(filter(lambda item: item is not None, spectrums_standards))
n_cleaned = len(spectrums_standards)
print(n_cleaned)
print(f"Filtered out {n_raw - n_cleaned} of a total of {n_raw} spectra. {n_cleaned} spectra remaining.")

import os
if os.path.exists("./testing/data/polyphenol-set1/spectra_set1.mgf"):
  os.remove("./testing/data/polyphenol-set1/spectra_set1.mgf")
if os.path.exists("./testing/data/polyphenol-set2/spectra_set2.mgf"):
  os.remove("./testing/data/polyphenol-set2/spectra_set2.mgf")

save_as_mgf(spectrums_exp, "./testing/data/polyphenol-set1/spectra_set1.mgf")
save_as_mgf(spectrums_standards, "./testing/data/polyphenol-set2/spectra_set2.mgf")

30
26
Filtered out 4 of a total of 30 spectra. 26 spectra remaining.
9
7
Filtered out 2 of a total of 9 spectra. 7 spectra remaining.


/opt/homebrew/Caskroom/miniconda/base/envs/ms2query/lib/python3.8/site-packages/pyteomics/auxiliary/file_helpers.py:593: FutureWarning: Opening an existing file in append mode. The default mode will change from 'a' to 'w' in a future version. Pass `file_mode='a'` to keep old behavior and suppress this warning.
  warnings.warn("Opening an existing file in append mode. "


In [3]:
experimental_spectra_folder_pos  = "./testing/data/polyphenol-set1"
experimental_spectra_filename_pos  = "./testing/data/polyphenol-set1/spectra_set1.mgf"
reference_standards_folder_pos = "./testing/data/polyphenol-set2"
reference_standards_filename_pos = "./testing/data/polyphenol-set2/spectra_set2.mgf"

models_and_library_folder_pos = "./testing/data/ms2query_models_and_library_pos"
models_and_library_folder_neg = "./testing/data/ms2query_models_and_library_neg"

results_folder = "./testing/results"

### Process and Annotate spectrum data

- An mgf file is loaded and matchms spectrum cleaning and harmonization functions are applied as a pipeline. 
- ms2query is run to generate classification tables and analog predictions. 
- The spectrum metadata is extracted as a pandas df.

Returns:
cleaned spectrum list, classification table, analog table, metadata table. All linked together through sample_idx.

In [4]:
from ms2query.run_ms2query import download_zenodo_files, run_complete_folder
from ms2query.ms2library import create_library_object_from_one_dir

/opt/homebrew/Caskroom/miniconda/base/envs/ms2query/lib/python3.8/site-packages/PubChemPy-1.0.4-py3.8.egg/pubchempy.py:563: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/homebrew/Caskroom/miniconda/base/envs/ms2query/lib/python3.8/site-packages/PubChemPy-1.0.4-py3.8.egg/pubchempy.py:563: SyntaxWarning: "is not" with a literal. Did you mean "!="?


In [5]:
# Specify zenodo doi's for positive and negative mode
zenodo_DOIs = {"positive": 6997924, "negative": 7107654}
# Downloads pretrained models and files for MS2Query (>2GB download)
download_zenodo_files(zenodo_DOIs["positive"], models_and_library_folder_pos)
download_zenodo_files(zenodo_DOIs["negative"], models_and_library_folder_neg)

# added all annotated GNPS pos csv to neg as well for inchikey to class

file with the name ./testing/data/ms2query_models_and_library_pos/ALL_GNPS_210409_positive_processed_annotated_CF_NPC_classes.txt already exists, so was not downloaded
file with the name ./testing/data/ms2query_models_and_library_pos/library_GNPS_15_12_2021_ms2ds_embeddings.pickle already exists, so was not downloaded
file with the name ./testing/data/ms2query_models_and_library_pos/library_GNPS_15_12_2021_s2v_embeddings.pickle already exists, so was not downloaded
file with the name ./testing/data/ms2query_models_and_library_pos/library_GNPS_15_12_2021.sqlite already exists, so was not downloaded
file with the name ./testing/data/ms2query_models_and_library_pos/ms2ds_model_GNPS_15_12_2021.hdf5 already exists, so was not downloaded
file with the name ./testing/data/ms2query_models_and_library_pos/ms2query_random_forest_model.pickle already exists, so was not downloaded
file with the name ./testing/data/ms2query_models_and_library_pos/spec2vec_model_GNPS_15_12_2021.model already exists,

In [6]:
# Create a MS2Library object
ms2library = create_library_object_from_one_dir(models_and_library_folder_pos)

/opt/homebrew/Caskroom/miniconda/base/envs/ms2query/lib/python3.8/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ms2query/lib/python3.8/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
2023-02-03 11:15:13.823413: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platf

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [7]:
# Run library search and analog search on your files.
run_complete_folder(
    ms2library, experimental_spectra_folder_pos, results_folder=results_folder)

AssertionError: Csv file location for results already exists

In [ ]:

#scores_s2v = compute_similarities_s2v(spectrums_exp, models_and_library_folder_pos)
#scores_cos = compute_similarities_cosine(spectrums_exp, type="ModifiedCosine")
#scores_ms2ds = compute_similarities_ms2ds(spectrums_exp, models_and_library_folder_pos)


In [8]:
from template_processing_funs import get_classes
import pandas as pd

In [9]:
def batch_run_get_classes(spectrum_list):
    """ Function queries GNPS API for classes for all spectra with inchi in spectrum list."""
    classes = []
    for iloc, spectrum in enumerate(spectrum_list):
        if iloc % 2 == 0 and not iloc == 0:
            print( ( f"{iloc} spectra done,",
                f" {len(spectrum_list) - (iloc+1)} spectra remaining."))
        inchi = spectrum.get("inchi")
        classes.append(get_classes(inchi))
    classes = pd.DataFrame.from_dict(classes)
    return classes


#classes_df_standards = batch_run_get_classes(spectrums_standards)


In [10]:
run_complete_folder(
    ms2library, reference_standards_folder_pos, results_folder=results_folder)

AssertionError: Csv file location for results already exists

In [11]:
import numpy as np
import pandas as pd

def expand_ms2query_results_table(results_table, n_spectra):
    # Construct complete index for all possible query_spectrum_nr entries
    new_index = pd.Index(np.arange(1, n_spectra + 1), name="query_spectrum_nr")
    # Superimpose the new index and reset index to iloc
    out_df = results_table.set_index("query_spectrum_nr")
    out_df = out_df.reindex(new_index).reset_index()
    # add iloc index column
    out_df["source_spectrum_mgf_iloc"] = out_df.index
    return out_df



#class_df_standards.set_index('query_spectrum_nr')

In [12]:

spectrums_standards = list(load_from_mgf(reference_standards_filename_pos))
spectrums_exp = list(load_from_mgf(experimental_spectra_filename_pos))

print( len(spectrums_exp) + len(spectrums_standards))

class_df_exp = pd.read_csv("testing/results/spectra_set1.csv")
class_df_standards = pd.read_csv("testing/results/spectra_set2.csv")
print( class_df_exp.shape[0] + class_df_standards.shape[0] )

class_df_standards = expand_ms2query_results_table(class_df_standards, len(spectrums_standards))
class_df_exp = expand_ms2query_results_table(class_df_exp, len(spectrums_exp))
# Current merge script
all_spectra = spectrums_exp + spectrums_standards # list addition

class_df_exp["is_standard"] = False
class_df_exp["exp_metadata"] = None
class_df_standards["is_standard"] = True
class_df_standards["standard_metadata"] = None

#all_class_table = pd.concat([class_df_exp, class_df_standards]).reset_index()
all_class_table = pd.merge(class_df_exp, class_df_standards, how = 'outer').reset_index()
all_class_table["specxplore_id"] = all_class_table.index
all_class_table


33
30


,index,query_spectrum_nr,ms2query_model_prediction,precursor_mz_difference,precursor_mz_query_spectrum,precursor_mz_analog,inchikey,spectrum_ids,analog_compound_name,retention_time,...,cf_subclass,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results,source_spectrum_mgf_iloc,is_standard,exp_metadata,standard_metadata,specxplore_id
0,0,1,0.7530,0.0002,149.0598,149.060,WBYWAXJHAXSJNI,304135.0,CINNAMIC ACID,NaN,...,Cinnamic acids,Cinnamic acids,Cinnamic acids and derivatives,Phenylpropanoids (C6-C3),Shikimates and Phenylpropanoids,0,False,None,NaN,0
1,1,2,0.8826,0.0003,165.0547,165.055,NGSWKAQJJWESNS,1615.0,p-coumaric acid,NaN,...,Hydroxycinnamic acids and derivatives,Hydroxycinnamic acids,Cinnamic acids and derivatives,Phenylpropanoids (C6-C3),Shikimates and Phenylpropanoids,1,False,None,NaN,1
2,2,3,0.8329,0.0006,180.0654,180.066,QIAFMBKCNZACKA,304050.0,HIPPURIC ACID,NaN,...,Benzoic acids and derivatives,Hippuric acids,Dipeptides,Small peptides,Amino acids and Peptides,2,False,None,NaN,2
3,3,4,0.8768,0.0001,181.0491,181.049,QAIPRVGONGVQAS,255081.0,Caffeic acid,NaN,...,Hydroxycinnamic acids and derivatives,Hydroxycinnamic acids,Cinnamic acids and derivatives,Phenylpropanoids (C6-C3),Shikimates and Phenylpropanoids,3,False,None,NaN,3
4,4,5,0.8699,0.0002,195.0652,195.065,KSEBMYQBYZTDHS,62728.0,ferulic acid,NaN,...,Hydroxycinnamic acids and derivatives,Hydroxycinnamic acids,Cinnamic acids and derivatives,Phenylpropanoids (C6-C3),Shikimates and Phenylpropanoids,4,False,None,NaN,4
5,5,6,0.8491,0.0001,195.0649,195.065,KSEBMYQBYZTDHS,62728.0,ferulic acid,NaN,...,Hydroxycinnamic acids and derivatives,Hydroxycinnamic acids,Cinnamic acids and derivatives,Phenylpropanoids (C6-C3),Shikimates and Phenylpropanoids,5,False,None,NaN,5
6,6,7,0.7235,0.0008,229.0492,229.050,NXJCRELRQHZBQA,113272.0,Citropen,NaN,...,NaN,Coumarins and derivatives,Simple coumarins,Coumarins,Shikimates and Phenylpropanoids,6,False,None,NaN,6
7,7,8,0.8798,0.0005,243.1015,243.101,ADFCQWZHKCXPAJ,25871.0,Equol,NaN,...,Isoflavans,Isoflavanols,Isoflavanones,Isoflavonoids,Shikimates and Phenylpropanoids,7,False,None,NaN,7
8,8,9,0.8714,0.0003,255.0647,255.065,GPGOCTLAUAHUQO,68703.0,"3,4'-Dihydroxyflavone",NaN,...,NaN,NaN,NaN,NaN,NaN,8,False,None,NaN,8
9,9,10,0.8720,0.0003,271.0597,271.060,TZBJGXHYKVUXJN,59874.0,genistein,NaN,...,Isoflav-2-enes,Isoflavones,Isoflavones,Isoflavonoids,Shikimates and Phenylpropanoids,9,False,None,NaN,9


### Process and Annotate reference standards

- An mgf file is loaded and matchms spectrum cleaning and harmonization functions are applied as a pipeline. 
- GNPS API interfacing code is used to run classyfire and npclassifier on inchi/smiles.
- The spectrum metadata is extracted as a pandas df.
- Any additonal spectrum metadata is added if available.
- All spectra are indexed by their standards_idx



To add any custom metadata require the following:
- mgf file contains unique spectrum identifier c_id
- csv file with additional metadata contains this same unique spectrum identifier c_id

Provide function for metadata table joining such that
[chemical classes table (with c_id column from mgf)] joined with [metadata table from csv from c_id]

### Merge spectral data

Sample data and reference standard data are combined together. Reference standards can be identified easily in post via their is_standard == True entry in the joint metadata table or via their inchi/smiles.

A new idx is generated to uniquely identify each spectrum in the merged data. Any otherwise useful spectrum ids will be within the metadata.

### Get Pairwise Similarities using matchms

Requires:model files and their paths, spectrum list
Returns:idx ordered pairwise similarities in np matrix format

Note:
- spec2vec and ms2deepscore come with their own tutorials on how to do this. 
- installations of both tools may be tricky depdending on the operating system
- matchms has a nice interface for this already; all we can do is wrap it away and limit it
- WARNING:all three similarity matrices are currently necessary for the dashboard; they cannot be missing.

Proposed Solution:

--> leave these steps in the original functions style and mainly provide output glue.
--> a wrapper function will add additional baggage and will only be handy if we can guarantee it'll run.

In [14]:
from template_processing_funs import compute_similarities_cosine, compute_similarities_ms2ds, compute_similarities_s2v
scores_s2v = compute_similarities_s2v(all_spectra, models_and_library_folder_pos)
scores_cos = compute_similarities_cosine(all_spectra, type="ModifiedCosine")
scores_ms2ds = compute_similarities_ms2ds(all_spectra, models_and_library_folder_pos)

2023-02-03 11:15:59,721:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-02-03 11:15:59,722:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-02-03 11:16:00,136:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-02-03 11:16:00,137:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-02-03 11:16:00,137:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-02-03 11:16:00,138:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-02-03 11:16:00,138:WARNING:matchms:ModifiedCosine:Precursor_mz must be of type int or float. Apply 'add_precursor_mz' filter first.
2023-02-03 11:16:00,138:WARNING:matchms:M

Calculating vectors of reference spectrums:   0%|          | 0/33 [00:00<?, ?it/s]2023-02-03 11:16:01.620060: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-03 11:16:01.724520: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 200ms/step


Calculating vectors of reference spectrums:   3%|▎         | 1/33 [00:00<00:08,  3.61it/s]

1/1 [==============================] - 0s 19ms/step


...

Calculating vectors of reference spectrums:  82%|████████▏ | 27/33 [00:01<00:00, 23.55it/s]

1/1 [==============================] - 0s 12ms/step


Calculating vectors of reference spectrums:  91%|█████████ | 30/33 [00:01<00:00, 24.57it/s]

1/1 [==============================] - 0s 13ms/step


Calculating vectors of reference spectrums: 100%|██████████| 33/33 [00:01<00:00, 20.03it/s]


### Run K-Medoid Clustering Grid

Here, K-Medoid clustering is run for many levels of K to achieve a good Silhouette score.

Idea: this particular code can be run and rerun easily; the grid can be modified until a suitable K is found.

Return: A classification table with suitable K clustering coefficients. Small K for broad trends, large K for granularity in the t-SNE embedding.

SyntaxError: invalid syntax (1347755551.py, line 1)

In [16]:
import kmedoids
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr, spearmanr  
import sklearn
import plotly.express as px
from sklearn.manifold import TSNE

In [17]:
dist = 1.- scores_ms2ds
# Deal with floating point issues
dist = np.round(dist, 4)
dist = np.clip(dist, a_min = 0, a_max = 1)

kclass_table = pd.DataFrame()
scores = []
n_clusters = np.arange(2, 30, 2).tolist()
for k in n_clusters:
    cluster = kmedoids.KMedoids(
        n_clusters=k, metric='precomputed', random_state=0, 
        method = "fasterpam")  
    cluster_km = cluster.fit_predict(dist)
    cluster_km = ["KM_" + str(elem) for elem in cluster_km]
    kclass_table[f"kmedoid_{k}"] = cluster_km
    scores.append(
        sklearn.metrics.silhouette_score(
            X = dist, labels = cluster_km, metric= "precomputed"))
kclass_table["specxplore_id"] = kclass_table.index
kclass_table.head()
print(list(kclass_table.columns))

['kmedoid_2', 'kmedoid_4', 'kmedoid_6', 'kmedoid_8', 'kmedoid_10', 'kmedoid_12', 'kmedoid_14', 'kmedoid_16', 'kmedoid_18', 'kmedoid_20', 'kmedoid_22', 'kmedoid_24', 'kmedoid_26', 'kmedoid_28', 'specxplore_id']


In [18]:
fig = px.scatter(x=n_clusters, y=scores)
fig.show()

In [19]:
km = "kmedoid_20"

### Run t-SNE Grid

Here, a t-SNE tuning round is done to assess what levels of perplexity would lead to good distance preservation properties of the embedding. Learning rate and number of iterations may also be investigated, but this tuning will be slower.

Speed depends on data size and settings. A single run may take a couple of minutes for large datasets and certain settings.

In [21]:
spearman_list = []
pearson_list = []
perplexities = [2,3,4,5,6,7,8,9,10,15,20,25]
for perplexity in perplexities:
    model = TSNE(
        metric="precomputed", random_state = 123, init = "random",
        perplexity = perplexity)
    z = model.fit_transform(dist)
    dist_tsne = squareform(pdist(z, 'seuclidean'))
    spearman_list.append(np.array(spearmanr(dist.flat, dist_tsne.flat))[0])
    pearson_list.append(np.array(pearsonr(dist.flat, dist_tsne.flat))[0])

In [22]:
fig = px.scatter(
    x=perplexities, y=spearman_list, title="Spearman vs Perplexity")
fig.show()
fig = px.scatter(
    x=perplexities, y=pearson_list, title="Pearson vs Perplexity")
fig.show()

In [23]:
model = TSNE(
    metric="precomputed", random_state = 123, init = "random",
    perplexity = 10)
z = model.fit_transform(dist)

In [24]:
tsnedf = pd.DataFrame()
tsnedf["clust"] = kclass_table[km]
tsnedf["x"] = z[:,0] * 50
tsnedf["y"] = z[:,1] * 50
fig = px.scatter(tsnedf, x="x", y="y", color="clust", hover_data=['clust'])
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,)
fig.show()

### Construct specXplore data structure

Construct a specXplore data structure for use within the dashboard. Essentially a class with named data entries to use. This avoids passing around many parameters at each step of the dashboard, and provides a single place to look at the data structure used throughout specXplore.

In [25]:
import specxplore.specxplore_data
from specxplore.specxplore_data import specxplore_data

is_standard = np.array(all_class_table["is_standard"])
spec_classes = all_class_table[['cf_kingdom', 'cf_superclass', 'cf_class', 'cf_subclass',
       'cf_direct_parent', 'npc_class_results', 'npc_superclass_results',
       'npc_pathway_results', 'specxplore_id']]
mz = [spec.get("precursor_mz") for spec in all_spectra]
specxplore_id = np.array(all_class_table["specxplore_id"])

phophe_specxplore = specxplore_data(
  scores_ms2ds,scores_s2v, scores_cos, tsnedf, kclass_table, spec_classes, 
  is_standard, all_spectra, mz, specxplore_id)

In [26]:
phophe_specxplore.class_table.columns

Index(['kmedoid_2', 'kmedoid_4', 'kmedoid_6', 'kmedoid_8', 'kmedoid_10',
       'kmedoid_12', 'kmedoid_14', 'kmedoid_16', 'kmedoid_18', 'kmedoid_20',
       'kmedoid_22', 'kmedoid_24', 'kmedoid_26', 'kmedoid_28',
       'specxplore_id'],
      dtype='object')

In [27]:
s = all_spectra[0]
s.peaks.mz
s.peaks.intensities

array([0.64292227, 1.        , 0.44513617, 0.62423902])

In [28]:

import pickle
with open('./testing/results/phophe_specxplore.pickle', 'wb') as file:
  pickle.dump(phophe_specxplore, file)